In [810]:
import pandas as pd
import numpy as np
import random
from scipy.io.arff import loadarff
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, ConcatDataset, SubsetRandomSampler, random_split

import torchvision
from torchvision import transforms

from torchmetrics import Accuracy
from torchmetrics.classification import BinaryAccuracy

In [419]:
CRICKET_X = 'CricketX'
INSECT_WINGBEAT_SOUND = 'InsectWingbeatSound'
U_WAVE_GESTURE_LIBRARY_ALL = 'UWaveGestureLibraryAll'
#EPILEPRIC_SEIZURE= 'EpilepticSeizure'
#MFPT = 'MFPT'
#XJTU = 'XJTU'

In [824]:
batch_size = 128
num_features = 64
num_epochs = 1000
patience = 200 # num of epochs with prev_loss - loss <= threshold without stopping
threshold = 0.0001
learning_rate = 0.01
alpha = 0.4 #past-future segment split ratio

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>def load_labeledDataframe()</h5></div>

Provides the labeled version of dataset.

Takes: 
- dataset: name of dataset (e.g. CRICKET_X)

Returns:
- df_train: pandas dataframe of train samples
- df_test: pandas dataframe of test samples
- num_classes: number of classes of dataset

In [707]:
def load_labeledDataframe(dataset = None):

    print(f'Dataset: {dataset}')

    #load dataset
    train = np.loadtxt(f'../Datasets/{dataset}/{dataset}_TRAIN.tsv').astype(np.float32)
    test = np.loadtxt(f'../Datasets/{dataset}/{dataset}_TEST.tsv').astype(np.float32)

    #divide dataset in attributes and labels
    X_train = train[:, 1:] # each row, columns from 1 to n
    y_train = np.squeeze(train[:, 0:1],axis=1).astype(np.int32) # each row, columns 0
    X_test = test[:, 1:] # each row, columns from 1 to n
    y_test= np.squeeze(test[:, 0:1],axis=1).astype(np.int32) # each row, columns 0

    #encodes labels between 0 and n-1
    encoder = preprocessing.LabelEncoder()
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    encoder.fit(y_test)
    y_test = encoder.transform(y_test)

    #create a list of name for attributes
    attributes = [f'att{i}' for i in range(len(X_train[0]))]

    #create a pandas dataframe for train set
    df_Xtrain = pd.DataFrame(X_train, columns=attributes)
    df_ytrain = pd.DataFrame(y_train, columns=['target'])
    df_train = pd.concat([df_Xtrain, df_ytrain], axis = 1)

    #create a pandas dataframe for test set
    df_Xtest = pd.DataFrame(X_test, columns=attributes)
    df_ytest = pd.DataFrame(y_test, columns=['target'])
    df_test = pd.concat([df_Xtest, df_ytest], axis = 1)

    num_classes = df_ytrain.nunique()[0]

    return df_train, df_test, num_classes

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>def load_unlabeledDataframe()</h5></div>

Provides the unlabeled version of dataset.

Takes: 
- dataset: name of dataset (e.g. CRICKET_X)

Returns:
- df_past: pandas dataframe of past segments
- df_pos: pandas dataframe of future segments in positive relation with past segments
- df_neg: pansas dataframe of future segments in negative relation with past segments

In [712]:
def load_unlabeledDataframe(dataset = None):

    print(f'Dataset: {dataset}')

    #load dataset
    train = np.loadtxt(f'../Datasets/{dataset}/{dataset}_TRAIN.tsv').astype(np.float32)

    #features only
    X_train = train[:, 1:] # each row, columns from 1 to n

    #X_train = np.array([np.array([1,2,3,4,5,6,7,8,9,10]), np.array([11,12,13,14,15,16,17,18,19,20])])

    len_train = len(X_train[1])
    len_past = int(len(X_train[1])*alpha)
    len_future = len_train - len_past
    roll_param = int(np.random.randint(len_future)*0.75) #roll 

    #np.roll shift all segement on right of roll_param positions

    #print(f'len(train): {len_train}, len_past: {len_past}, len_future: {len_future}, roll_param: {roll_param}')

    past = X_train[:, 0:len_past] #past segments (past) form 0 to len_past-1
    future_pos = X_train[:, len_past:len_train] #future positive segments from len_past to len_train
    future_neg = np.roll(X_train[:, len_past:len_train], roll_param) #future negative segments form len_past to len_train, and roll


    #create a list of name for attributes past, future_positive and future_negative segments
    att_past = [f'past{i}' for i in range(len_past)]
    att_pos = [f'pos{i}' for i in range(len_future)]
    att_neg = [f'neg{i}' for i in range(len_future)]

    #create a pandas dataframe for past, future_positive and future_negative segments
    df_past = pd.DataFrame(past, columns=att_past)
    df_pos = pd.DataFrame(future_pos, columns=att_pos)
    df_neg = pd.DataFrame(future_neg, columns=att_neg)

    return df_past, df_pos, df_neg

In [710]:
df_train, df_test, num_classes = load_labeledDataframe(dataset=CRICKET_X)

Dataset: CricketX


In [713]:
df_past, df_pos, df_neg = load_unlabeledDataframe(dataset=CRICKET_X)

Dataset: CricketX


In [421]:
# def magnitude_warp_s(x, sigma=0.2, knot=4, plot=False):

#     print(f'x.shape: {x.shape}')
#     print(f'x: {x}')
#     print(f'np.ones((x.shape[1], 1)).shape: {np.ones((x.shape[1], 1)).shape}')
#     print(f'np.ones((x.shape[1], 1)): {np.ones((x.shape[1], 1))}')
#     print(f'np.linspace(0, x.shape[0] - 1., num=knot + 2)).shape: {np.linspace(0, x.shape[0] - 1., num=knot + 2).shape}')
#     print(f'np.linspace(0, x.shape[0] - 1., num=knot + 2)): {np.linspace(0, x.shape[0] - 1., num=knot + 2)}')

#     from scipy.interpolate import CubicSpline
#     orig_steps = np.arange(x.shape[0])

#     random_warps = np.random.normal(loc=1.0, scale=sigma, size=(1, knot + 2, x.shape[1]))
#     warp_steps = (np.ones((x.shape[1], 1)) * (np.linspace(0, x.shape[0] - 1., num=knot + 2))).T

#     print(f'warp_steps: {warp_steps}')

#     li = []
#     for dim in range(x.shape[1]):
#         li.append(CubicSpline(warp_steps[:, dim], random_warps[0, :, dim])(orig_steps))
#     warper = np.array(li).T

#     x_ = x * warper

#     #if plot:
#     #    hlp.plot1d(x, x_, save_file='aug_examples/magnitude_warp_s.png')
    
#     return x_


# def time_warp_s(x, sigma=0.2, knot=4, plot=False):
#     from scipy.interpolate import CubicSpline
#     orig_steps = np.arange(x.shape[0])

#     random_warps = np.random.normal(loc=1.0, scale=sigma, size=(1, knot + 2, x.shape[1]))
#     warp_steps = (np.ones((x.shape[1], 1)) * (np.linspace(0, x.shape[0] - 1., num=knot + 2))).T

#     ret = np.zeros_like(x)
#     for dim in range(x.shape[1]):
#         time_warp = CubicSpline(warp_steps[:, dim],
#                                 warp_steps[:, dim] * random_warps[0, :, dim])(orig_steps)
#         scale = (x.shape[0] - 1) / time_warp[-1]
#         ret[:, dim] = np.interp(orig_steps, np.clip(scale * time_warp, 0, x.shape[0] - 1),
#                                    x[:, dim]).T
#     #if plot:
#     #    hlp.plot1d(x, ret, save_file='aug_examples/time_warp_s.png')
    
#     return ret


# class MagnitudeWrap:
#     def __init__(self, sigma, knot, p):
#         self.sigma = sigma
#         self.knot = knot
#         self.p = p

#     def __call__(self, data):
#         print('### MagnitudeWrap')

#         if random.random() < self.p:
#             return self.forward(data)

#         return data

#     def forward(self, data):
#         return magnitude_warp_s(data, sigma=self.sigma, knot=self.knot)


# class TimeWarp:
#     def __init__(self, sigma, knot, p):
#         self.sigma = sigma
#         self.knot = knot
#         self.p = p

#     def __call__(self, data):
#         print('### TimeeWrap')

#         if random.random() < self.p:
#             return self.forward(data)

#         return data

#     def forward(self, data):
#         return time_warp_s(data, sigma=self.sigma, knot=self.knot)

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>class LabeledDataset()</h5></div>

Class for labeled dataset. 

Takes: 
- dataframe: pandas dataframe of dataset

Methods:
- len: returns the length of dataframe
- getitem: returns the pair (feature, tarhet) of sample at index 'idx' of dataframe

In [726]:
class LabeledDataset(Dataset):
    def __init__(self, dataframe=None):

        self.df = dataframe

        #self.magnitude_warp = MagnitudeWrap(sigma=0.3, knot=4, p=0.3)
        #self.time_warp = TimeWarp(sigma=0.2, knot=8, p=0.3)
        #self.transform = transform = transforms.Compose([self.magnitude_warp, self.time_warp])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        self.target = torch.from_numpy(np.array(self.df['target'][idx])).to(torch.int32)
        self.features = torch.from_numpy(np.array(self.df.iloc[idx:idx+1, :len(self.df.columns)-1])).to(torch.float32)

        #self.features = self.transform(self.features)

        return  self.features, self.target

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>class UnlabeledDataset()</h5></div>

Class for unlabeled dataset. 

Takes: 
- df_past: pandas dataframe of past segments
- df_pos: pandas dataframe of future segments in positive relation with past segments
- df_neg: pansas dataframe of future segments in negative relation with past segments

Methods:
- len: returns the length of dataframe (df_past, df_pos and df_neg have same length) 
- getitem: returns three samples, each at index 'idx' of df_past, df_pos and df_neg 

In [721]:
class UnlabeledDataset(Dataset):
    def __init__(self, df_past=None, df_pos=None, df_neg=None):

        self.df_past = df_past
        self.df_pos = df_pos
        self.df_neg = df_neg

        #self.magnitude_warp = MagnitudeWrap(sigma=0.3, knot=4, p=0.3)
        #self.time_warp = TimeWarp(sigma=0.2, knot=8, p=0.3)
        #self.transform = transform = transforms.Compose([self.magnitude_warp, self.time_warp])

    def __len__(self):
        return len(self.df_past) # len is the same for all df

    def __getitem__(self, idx):
        self.features_past = torch.from_numpy(np.array(self.df_past.iloc[idx:idx+1,:])).to(torch.float32)
        self.features_pos = torch.from_numpy(np.array(self.df_pos.iloc[idx:idx+1,:])).to(torch.float32)
        self.features_neg = torch.from_numpy(np.array(self.df_neg.iloc[idx:idx+1,:])).to(torch.float32)

        #self.features = self.transform(self.features)

        return  self.features_past, self.features_pos, self.features_neg

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>class BackboneEncoder()</h5></div>

Implementation of Backbone Encoder model. This model is used for features extraction of samples.

Takes: 
- num_features: number of features in output by BackboneEncoder

Input:
- tensor of shape [batch_size, 1, num_samples]

Output:
- tensor of shape [batch_size, num_features]

In [546]:
class BackboneEncoder(nn.Module):
    def __init__(self, num_features=None):
        super().__init__()
        self.num_features = num_features

        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm1d(num_features=8),
            nn.ReLU()
        )

        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels=8, out_channels=16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm1d(num_features=16),
            nn.ReLU()
        )

        self.layer3 = nn.Sequential(
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm1d(num_features=32),
            nn.ReLU()
        )

        self.layer4 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm1d(num_features=self.num_features),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(output_size=1)
        )

        self.flatten = nn.Flatten()


    def forward(self, x):
        
        x = x.view(x.shape[0], 1, -1) # shape = (batch_size, 1, num_col)

        x = self.layer1(x)
        #print(f'layer1: {x.shape}')
        x = self.layer2(x)
        #print(f'layer2: {x.shape}')
        x = self.layer3(x)
        #print(f'layer3: {x.shape}')
        x = self.layer4(x)
        #print(f'layer4: {x.shape}')
        x = self.flatten(x)
        #print(f'flatten: {x.shape}')
        x = F.normalize(x, dim=1)
        #print(f'normalize: {x.shape}')
        
        return x

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>class ClassificationHead()</h5></div>

Implementation of Classification Head model. This model is used for multiclass classification samples in labeled dataset. Is lies after the Backbone Encoder, which has [batch_size, num_features] as output shape.

Takes: 
- num_features: number of features in output by BackboneEncoder

Input:
- tensor of shape [batch_size, num_features]

Output:
- tensor of shape [batch_size, num_classes]

In [545]:
class ClassificationHead(nn.Module):
    def __init__(self, num_features=None, num_classes=None):
        super().__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        self.fc = nn.Linear(self.num_features, self.num_classes)

    def forward(self, x):
        return self.fc(x)

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>class RelationHead()</h5></div>

Implementation of Relation Head model. This model is used for binary classification of relation between past-future segments. Is lies after the Backbone Encoder, which has [batch_size, num_features] as output shape.

Takes: 
- num_features: number of features in output by BackboneEncoder

Input:
- tensor of shape [batch_size, num_features]

Output:
- tensor of shape [batch_size, 1]

In [544]:
class RelationHead(nn.Module):
    def __init__(self, num_features=None):
        super().__init__()
        self.num_features = num_features
        self.fc1 = nn.Linear(2*self.num_features, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.leakyReLU = nn.LeakyReLU()
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):

        x = self.fc1(x)
        x = self.bn1(x)
        x = self.leakyReLU(x)
        x = self.fc2(x)
        
        return x

In [785]:
labeledDataset = LabeledDataset(dataframe=df_train)
labeledTrain = DataLoader(dataset=labeledDataset, batch_size=128, shuffle=True)
itl = iter(labeledTrain)
xl = next(itl)[0]
print(xl.shape)

torch.Size([128, 1, 300])


In [796]:
unlabeledDataset = UnlabeledDataset(df_past=df_past, df_pos=df_pos, df_neg=df_neg)
unlabeledTrain = DataLoader(dataset=unlabeledDataset, batch_size=128, shuffle=True)
itu = iter(unlabeledTrain)
xu = next(itu)

print(xu[0].shape)
print(xu[1].shape)
print(xu[2].shape)

torch.Size([128, 1, 120])
torch.Size([128, 1, 180])
torch.Size([128, 1, 180])


In [797]:
backboneEncoder = BackboneEncoder(num_features=num_features)
classificationHead = ClassificationHead(num_features=num_features, num_classes=num_classes)
relationHead = RelationHead(num_features=num_features)

In [798]:
ol1 = backboneEncoder(xl)
ol2 = classificationHead(ol1)
print(ol2.shape)

torch.Size([128, 12])


In [825]:
ou1 = backboneEncoder(xu[0])
ou2 = backboneEncoder(xu[1])
ou3 = backboneEncoder(xu[2])

print(ou1.shape)
print(ou2.shape)
print(ou3.shape)

xu1 = torch.cat((ou1, ou2), dim=1)
xu2 = torch.cat((ou1, ou3), dim=1)

print(xu1.shape)
print(xu2.shape)

ou4 = relationHead(xu1)
ou5 = relationHead(xu2)

print(ou4.shape)
print(ou5.shape)

xu3 = torch.cat((ou4, ou5), dim=0)

print(xu3.shape)

torch.Size([128, 64])
torch.Size([128, 64])
torch.Size([128, 64])
torch.Size([128, 128])
torch.Size([128, 128])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([256, 1])


<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>def accuracy()</h5></div>

Implements accuracy metrics, relying on torchmetrics.Accuracy and torchmetrics.classification.BinaryAccuracy methods.

Takes: 
- predictions: predictions makes by model
- targets: labels of samples in dataset
- num_classes: number of classes of dataset

Returns:
- accuracy score

In [812]:
def accuracy(predictions=None, targets=None, num_classes=None):
    if num_classes == 2:
        accuracy = BinaryAccuracy()
    else:    
        accuracy = Accuracy(task='multiclass', num_classes=num_classes)
        
    return accuracy(predictions, targets)

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>class EarlyStopping()</h5></div>

Implementation of early stopping technique.

Takes: 
- patience: number of epochs with abs(prev_loss-loss) <= threshold without stopping, where 'prev_loss' is the loss recordered in the previous epoch and 'loss' is the loss computed in the current epoch
- threshold: below this number there isn't learning between previous epoch and current epoch

Returns:
- True: if the counter has reached the number 'patience'
- False: otherwise

Method:
- init: initializes counter=0 and prev_loss=0
- check: checks if 'counter' has reached the number 'patience' or update it.

In [822]:
class EarlyStopping():
    def __init__(self, patience=None, threshold=None):
        super().__init__()
        self.patience=patience #number of epochs with abs(prev_loss-loss) <= threshold without stopping
        self.threshold=threshold 
        self.counter=0 #counter of epochs with abs(prev_loss-loss) <= threshold
        self.prev_loss=0 #loss of previous epoch
    
    def __check__(self, loss=None):

        if self.counter==0: #if counter == 0 -> first epoch
            self.prev_loss=loss
            return False

        elif self.counter==patience: #if counter == patience -> stop 
            return True

        else:   # else update counter or initialize it to 0
            if abs(self.prev_loss-loss) <= self.threshold:
                self.counter+=1
            else:
                self.counter=0
            return False

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>def model_crt_opt()</h5></div>

Provides models, criterions, and optimizer.

Takes: 
- device: cpu/gpu

Returns:
- models: backbone encoder, classification head, relation head
- criterions: cross entropy loss (for supervised learning), binary cross entropy loss (for self-supervised learning)
- optimizer: Adam

In [801]:
def model_crt_opt(device=None):

    backboneEncoder = BackboneEncoder(num_features=num_features).to(device)
    classificationHead = ClassificationHead(num_features=num_features, num_classes=num_classes).to(device)
    relationHead = RelationHead(num_features=num_features).to(device)

    ce = nn.CrossEntropyLoss().to(device)
    bce = nn.BCELoss().to(device)
    opt = torch.optim.Adam([{'backboneParams':backboneEncoder.parameters(), 
                            'clfHeadParams':classificationHead.parameters(),
                            'rltHeadParams':relationHead.parameters()}], lr=learning_rate)

    return backboneEncoder, classificationHead, relationHead, ce, bce, opt

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>def supervised_learning_epoch()</h5></div>
Implements supervised learning algorithm for one epoch.

Takes: 
- training: flag to choose between training and evaluation phase
- num_classes: number of classes of dataset (used for compute accuracy)
- loader: dataloader (from torch). 
- backboneEncoder: BackboneEncoder model for features extracting 
- classificationHead: classificationHead model for classification
- crossEntropy: cross entropy loss (from torch)
- optimizer: any kind of optimizer (eg. Adam from torch)
- device: cpu/gpu

Returns:
- loss: loss of this epoch
- acc: accuracy of this epoch

In [818]:
def supervised_learning_epoch(training=None, num_classes=None, loader=None, backboneEncoder=None, 
                            classificationHead=None, crossEntropy=None, optimizer=None, device=None):
    
    total_loss = 0
    total_acc = 0

    if training:
        backboneEncoder.train()
        classificationHead.train()

    else:
        backboneEncoder.eval()
        classificationHead.eval()

    for i, data_lab in enumerate(loader):
        features, targets = data_lab
        features, targets = features.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = backboneEncoder(features)
        outputs = classificationHead(outputs)

        loss = crossEntropy(outputs, targets)

        if training:
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        total_acc += accuracy(predictions=outputs, targets=targets, num_classes=num_classes)

    loss = total_loss/len(loader)
    acc = total_acc/len(loader)

    return loss, acc

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>def selfsupervised_learning_epoch()</h5></div>
Implements self-supervised learning algorithm for one epoch.

Takes: 
- training: flag to choose between training and evaluation phase
- loader: dataloader (from torch). 
- backboneEncoder: BackboneEncoder model for features extracting 
- relationHead: relationHead model for binary classification of past-future segments
- binaryCrossEntropy: binary cross entropy loss (from torch)
- optimizer: any kind of optimizer (eg. Adam from torch)
- device: cpu/gpu

Returns:
- loss: loss of this epoch
- acc: accuracy of this epoch

In [820]:
def selfsupervised_learning_epoch(training=None, loader=None, backboneEncoder=None, relationHead=None, 
                                binaryCrossEntropy=None, optimizer=None, device=None):
    
    total_loss = 0
    total_acc = 0

    if training:
        backboneEncoder.train()
        relationHead.train()

    else:
        backboneEncoder.eval()
        relationHead.eval()


    for i, data in enumerate(loader):
        past_features, pos_features, neg_features = data
        past_features, pos_features, neg_features = past_features.to(device), pos_features.to(device), neg_features.to(device)

        optimizer.zero_grad()

        output_past = backboneEncoder(past_features)
        output_pos = backboneEncoder(pos_features)
        output_neg = backboneEncoder(neg_features)
        
        pos_segments = torch.cat((output_past, output_pos), dim=1)
        neg_segments = torch.cat((output_past, output_neg), dim=1)

        output_pos = relationHead(pos_segments)
        output_neg = relationHead(neg_segments)

        target_pos = torch.ones(output_pos.shape).to(torch.int32)
        target_neg = torch.zeros(output_neg.shape).to(torch.int32)

        outputs = torch.cat((output_pos, output_neg), dim=0)
        targets = torch.cat((target_pos, target_neg), dim=0)

        loss = binaryCrossEntropy(outputs, targets)

        if training:
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        total_acc += accuracy(predictions=outputs, targets=targets, num_classes=2)

    loss = total_loss/len(loader)
    acc = total_acc/len(loader)

    return loss, acc

<div style="background-color: rgb(0, 255, 255, 0.2);"><h5>def training()</h5></div>
Implements training algorithm. Computes the supervised training and self-supervised training for all epochs.

Takes: 
- labeledDataset: dataset with labels for supervised traning
- num_classes: number of classes of labeledDataset
- unlabeledDataset: dataset without labels for self-supervised traning
- backboneEncoder: BackboneEncoder model for features extracting 
- classificationHead: classificationHead model for classification
- relationHead: relationHead model for binary classification of past-future segments
- crossEntropy: cross entropy loss (from torch)
- binaryCrossEntropy: binary cross entropy loss (from torch)
- optimizer: any kind of optimizer (eg. Adam from torch)
- device: cpu/gpu

Returns:
- train_scores: tuple of 4 numpy arrays:
  - total_train_loss_sl: numpy array that contains all loss scores recorderd during supervised training
  - total_train_loss_ssl: numpy array that contains all loss scores recorderd during self-supervised training
  - total_train_acc_sl: numpy array that contains all accuracy scores recorderd during supervised training
  - total_train_acc_ssl: numpy array that contains all accuracy scores recorderd during self-supervised training
- eval_scores: tuple of 4 numpy arrays:
  - total_val_loss_sl: numpy array that contains all loss scores recorderd during supervised evaluation
  - total_val_loss_ssl: numpy array that contains all loss scores recorderd during self-supervised evaluation
  - total_val_acc_sl: numpy array that contains all accuracy scores recorderd during supervised evaluation
  - total_val_acc_ssl: numpy array that contains all accuracy scores recorderd during self-supervised evaluation

In [817]:
def training(labeledDataset=None, num_classes=None, unlabeledDataset=None,
            backboneEncoder=None, classificationHead=None, relationHead=None,
            crossEntropy=None, binaryCrossEntropy=None, optimizer=None, device=None):

    # sl: supervised learning
    # ssl: self-supervised learning

    len_train = int(np.floor((2/3)*len(labeledDataset)))
    len_val = int(np.ceil((1/3)*len(labeledDataset)))
    train_set_lab, validation_set_lab = random_split(labeledDataset, [len_train, len_val])
    train_set_unlab, validation_set_unlab = random_split(unlabeledDataset, [len_train, len_val])
    
    train_loader_lab = DataLoader(dataset=train_set_lab, batch_size=batch_size, shuffle=True)
    val_loader_lab = DataLoader(dataset=validation_set_lab, batch_size=batch_size, shuffle=True)
    train_loader_unlab = DataLoader(dataset=train_set_unlab, batch_size=batch_size, shuffle=True)
    val_loader_unlab = DataLoader(dataset=validation_set_unlab, batch_size=batch_size, shuffle=True)

    earlyStopping_sl = EarlyStopping(patience=patience, threshold=threshold)
    earlyStopping_ssl = EarlyStopping(patience=patience, threshold=threshold)

    total_train_loss_sl = np.array([])
    total_train_loss_ssl = np.array([])
    total_train_acc_sl = np.array([])
    total_train_acc_ssl = np.array([])

    total_val_loss_sl = np.array([])
    total_val_loss_ssl = np.array([])
    total_val_acc_sl = np.array([])
    total_val_acc_ssl = np.array([])

    for k in range(num_epochs):

        #TRAINING

        train_loss_sl, train_acc_sl = supervised_learning_epoch(training=True, num_classes=num_classes, loader_lab=train_loader_lab, 
                                                                backboneEncoder=backboneEncoder, classificationHead=classificationHead, 
                                                                crossEntropy=crossEntropy, optimizer=optimizer, device=device)

        train_loss_ssl, train_acc_ssl = selfsupervised_learning_epoch(training=True, loader_lab=train_loader_unlab, 
                                                                    backboneEncoder=backboneEncoder, relationHead=relationHead, 
                                                                    binaryCrossEntropy=binaryCrossEntropy, optimizer=optimizer, device=device)


        #VALIDATION

        val_loss_sl, val_acc_sl = supervised_learning_epoch(training=False, num_classes=num_classes, loader_lab=val_loader_lab, 
                                                            backboneEncoder=backboneEncoder, classificationHead=classificationHead, 
                                                            crossEntropy=crossEntropy, optimizer=optimizer, device=device)

        val_loss_ssl, val_acc_ssl = selfsupervised_learning_epoch(training=False, loader_lab=val_loader_unlab, 
                                                                    backboneEncoder=backboneEncoder, relationHead=relationHead, 
                                                                    binaryCrossEntropy=binaryCrossEntropy, optimizer=optimizer, device=device)
        
        total_train_loss_sl.append(train_loss_sl)
        total_train_loss_ssl.append(train_loss_ssl)
        total_train_acc_sl.append(train_acc_sl)
        total_train_acc_ssl.append(train_acc_ssl)

        total_val_loss_sl.append(val_loss_sl)
        total_val_loss_ssl.append(val_loss_ssl)
        total_val_acc_sl.append(val_acc_sl)
        total_val_acc_ssl.append(val_acc_ssl)
        
        if earlyStopping_sl.__check__(loss=val_loss_sl) and earlyStopping_ssl.__check__(loss=val_loss_ssl):
            break

    train_loss_sl = total_train_loss_sl.mean()
    train_loss_ssl = total_train_loss_ssl.mean()
    train_acc_sl = total_train_acc_sl.mean()
    train_acc_ssl = total_train_acc_ssl.mean()

    val_loss_sl = total_val_loss_sl.mean()
    val_loss_ssl = total_val_loss_ssl.mean()
    val_acc_sl = total_val_acc_sl.mean()
    val_acc_ssl = total_val_acc_ssl.mean()

    train_scores = (total_train_loss_sl, total_train_loss_ssl, total_train_acc_sl, total_train_acc_ssl)
    eval_scores = (total_val_loss_sl, total_val_loss_ssl, total_val_acc_sl, total_val_acc_ssl)
        
    return train_scores, eval_scores

In [752]:
training(labeledDataset=labeledDataset, unlabeledDataset=unlabeledDataset, backboneEncoder=backboneEncoder, classificationHead=classificationHead, relationHead=relationHead)